# Implementação do Zero

In [3]:
import numpy as np
from collections import Counter

In [4]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature =  feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


In [19]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None





    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)

    
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features =  X.shape
        n_labels = len(np.unique(y))

        #check stopping criteria. If achieved, return a leaf node
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)

        feat_idx = np.random.choice(n_features, self.n_features, replace=False)

        #find best split
        best_feature, best_thresh = self._best_split(X, y, feat_idx)

        #create child nodes
        left_idx, right_idx = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idx, :], y[left_idx], depth+1)
        right = self._grow_tree(X[right_idx, :], y[right_idx], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idx):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat in feat_idx:
            x_column = X[:, feat]
            thresholds = np.unique(x_column) #get unique values in the column

            for thresh in thresholds:
                #Calculate the informartion gain
                gain = self._information_gain(y, x_column, thresh)
                
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat
                    split_thresh = thresh

        return split_idx, split_thresh


    def _information_gain(self, y, x_column, thresh):
        #parent entropy
        parent_entropy =  self._entropy(y)

        #create the children
        left_idx, right_idx = self._split(x_column, thresh)

        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0

        #entropy of the children
        n =  len(y)
        n_left, n_right = len(left_idx), len(right_idx)
        e_left =  self._entropy(y[left_idx])
        e_right = self._entropy(y[right_idx])

        #calculate the information gain
        IG = parent_entropy - (n_left/n) * e_left - (n_right/n) * e_right
        return IG


    def _split(self, x_column, thresh):
        left_idx = np.argwhere(x_column <= thresh).flatten()
        right_idx = np.argwhere(x_column > thresh).flatten()
        return left_idx, right_idx


    def _entropy(self, y):
        hist = np.bincount(y)
        p_X = hist / len(y)
        entropy = -np.sum([p * np.log2(p) for p in p_X if p > 0])
        return entropy


    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]
        





    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
            
        return self._traverse_tree(x, node.right)


    def draw(self):
        self._draw_tree(self.root)


# Teste

In [20]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)


clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

acc = np.sum(predictions == y_test)/len(y_test)
print(acc)


0.9210526315789473
7 <= 0.05102
20 <= 16.77
13 <= 34.37
21 <= 33.17
1
21 <= 33.37
0
1
18 <= 0.02418
22 <= 108.4
0
1
1
1 <= 19.46
5 <= 0.08549
1
0
0
23 <= 861.5
21 <= 25.47
7 <= 0.07857
7 <= 0.05246
0
1
0
28 <= 0.2666
1
0
0


# Modelo Nativo do SKLearn

In [7]:
from sklearn.tree import DecisionTreeClassifier

sk_clf = DecisionTreeClassifier()
sk_clf.fit(X_train, y_train)
sk_predictions = sk_clf.predict(X_test)

sk_acc = np.sum(sk_predictions == y_test)/len(y_test)
print(sk_acc)


0.9122807017543859


# Expansão para Random Forest

In [17]:
class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_features=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split, max_depth=self.max_depth, n_features=self.n_features)
            X_random, y_random = self._bootstrap_sample(X, y)
            tree.fit(X_random, y_random)
            self.trees.append(tree)

    def  _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]


    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        y_pred = [self._most_common_label(pred) for pred in tree_preds]
        return np.array(y_pred)

In [18]:
RFclf = RandomForest(n_trees=5)
RFclf.fit(X_train, y_train)

RFpredictions = RFclf.predict(X_test)

RFacc = np.sum(RFpredictions == y_test)/len(y_test)
print(acc)

0.9122807017543859
